In [4]:
import pandas as pd
from datetime import datetime

Df = pd.read_csv("DatawithFSA.csv")

Df = Df[['fire_location_longitude','fire_location_latitude','temperature','relative_humidity','wind_speed','fire_start_date','ex_hectares']]

def getDateandTimeInfo(date):
    # returns specific time of a day as a decimal and the date as a number
    
    dayspast = [31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]

    if pd.isna(date):
        return (0,0)
    elif date[1] == "/" or date[2] == "/":
        listTotal = date.split(" ")
        listTime = listTotal[1].split(":")
        listDate = listTotal[0].split("/")
        m = int(listDate[0])
        d = int(listDate[1])
        h = int(listTime[0])
        min = int(listTime[1])
        datenum = dayspast[m-1]+d
        timenum = h*60 + min
        return (datenum, timenum)
    else:
        convDate = datetime.strptime(date, "%Y-%m-%d %H:%M")
        startofyear = datetime(convDate.year, 1, 1)
        datenum = (convDate - startofyear).days
        timenum = convDate.hour * 60 + convDate.minute
        return (datenum, timenum)

toDrop = []
cnt = 0
for series_name,series in Df.items():
    for i in reversed(range(len(series)-cnt)):
        if pd.isna(series[i]):
            toDrop.append(i)

toDrop = sorted(list(set(toDrop)),reverse=True)
for i in toDrop:
    Df = Df.drop(index = i)

Df=Df.reset_index()
Df=Df.drop('index', axis = 'columns')

for series_name,series in Df.items():
    print(series_name)
    cnt=0
    for i in reversed(range(len(series))):
        if pd.isna(series[i]):
            cnt+=1
    print(cnt)


maxlist = []
adjusted_date = []
adjusted_time = []

#'fire_start_date'
for i in Df['fire_start_date']:
    timeList = getDateandTimeInfo(i)
    adjusted_date.append(timeList[0])
    adjusted_time.append(timeList[1])

Df.insert(loc=len(Df.columns)-1, column ="data_value", value = adjusted_date)
Df.insert(loc=len(Df.columns)-1, column='time_value', value = adjusted_time)
Df = Df.drop(columns="fire_start_date")
print(Df)




fire_location_longitude
0
fire_location_latitude
0
temperature
0
relative_humidity
0
wind_speed
0
fire_start_date
0
ex_hectares
0
       fire_location_longitude  fire_location_latitude  temperature  \
0                  -119.926971               59.522139         28.8   
1                  -116.118960               52.692262          2.0   
2                  -110.719950               55.937050         16.0   
3                  -111.160750               56.425933          7.0   
4                  -116.058633               53.693450         12.8   
...                        ...                     ...          ...   
20167              -114.823717               51.158683          4.0   
20168              -114.869317               51.161883          8.0   
20169              -115.392633               51.084150          1.0   
20170              -114.997667               51.333900         -1.0   
20171              -115.316300               51.071467          3.0   

       relative_h

In [11]:
# stores data in format (max, min)
maxMinDict = {}
NormalizedDf = pd.DataFrame()

for i in Df.columns:
    if i == "ex_hectares":
        NormalizedDf[i] = Df[i]
    else:
        curDf = Df[i]
        minValue = curDf.min()
        maxValue = curDf.max()
        maxMinDict[i] = (minValue,maxValue)

        normData = (curDf - minValue)/(maxValue-minValue)
        NormalizedDf[i] = normData

NormalizedDf = NormalizedDf.sample(frac=1)
NormalizedDf= NormalizedDf.reset_index()
NormalizedDf= NormalizedDf.drop('index', axis = 'columns')

print(NormalizedDf)


       fire_location_longitude  fire_location_latitude  temperature  \
0                     0.442490                0.864699     0.867824   
1                     0.856002                0.695155     0.507343   
2                     0.928202                0.630819     0.587450   
3                     0.457936                0.847448     0.807744   
4                     0.116746                0.537298     0.747664   
...                        ...                     ...          ...   
20167                 0.042822                0.688352     0.627503   
20168                 0.196952                0.777739     0.801068   
20169                 0.559872                0.860429     0.734312   
20170                 0.550456                0.624446     0.734312   
20171                 0.479933                0.232059     0.707610   

       relative_humidity  wind_speed  data_value  time_value  ex_hectares  
0                   0.22    0.111111    0.408163    0.528145       121.

In [15]:
divider = int(0.2*len(NormalizedDf))
testDf = NormalizedDf[:divider]
trainDf = NormalizedDf[divider:]

testDf.to_csv("./mlData/TestData.csv")
trainDf.to_csv("./mlData/TrainData.csv")

print(maxMinDict)

{'fire_location_longitude': (-119.999983, -110.000917), 'fire_location_latitude': (48.998195, 59.99951), 'temperature': (-35.0, 39.9), 'relative_humidity': (0.0, 100.0), 'wind_speed': (0.0, 90.0), 'data_value': (4, 396), 'time_value': (0, 1439)}
